## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [14]:
freqs_ted = {}
for w in [item for sublist in sentences_ted for item in sublist]:
    if w in freqs_ted:
        freqs_ted[w] += 1
    else:
        freqs_ted[w] = 1
        
words_top_ted = sorted(freqs_ted, key=lambda x: int(freqs_ted[x]), reverse=True)
counts_ted_top1000 = [freqs_ted[word] for word in words_top_ted[:1000]]

print(words_top_ted[:10])
print(counts_ted_top1000[:10])

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you']
[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923]


Plot distribution of top-1000 words

In [15]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [16]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.models import Word2Vec

In [22]:
model_ted = gensim.models.Word2Vec(min_count = 5, size=100, sentences = sentences_ted)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [23]:
model_ted.most_similar("woman")

[('man', 0.8427544236183167),
 ('girl', 0.8252713680267334),
 ('lady', 0.8101569414138794),
 ('boy', 0.7733271718025208),
 ('child', 0.7629806995391846),
 ('kid', 0.731449544429779),
 ('soldier', 0.7185887694358826),
 ('gentleman', 0.7180517911911011),
 ('nurse', 0.7110512256622314),
 ('teacher', 0.697555661201477)]

In [24]:
model_ted.most_similar("laptop")

[('smartphone', 0.7353024482727051),
 ('cellphone', 0.7339715957641602),
 ('bedroom', 0.7286555767059326),
 ('desktop', 0.7175372838973999),
 ('backpack', 0.7084414958953857),
 ('score', 0.7078048586845398),
 ('wallet', 0.7065618634223938),
 ('pocket', 0.6980549097061157),
 ('card', 0.6966496109962463),
 ('schedule', 0.6939713358879089)]

In [25]:
model_ted.most_similar(positive=['cat', 'fish'], negative=['dog'])

[('shrimp', 0.6931604146957397),
 ('meat', 0.6690148115158081),
 ('tuna', 0.6618388295173645),
 ('fruit', 0.6532405614852905),
 ('algae', 0.6452613472938538),
 ('holes', 0.644997775554657),
 ('coral', 0.6417567729949951),
 ('jellyfish', 0.6398381590843201),
 ('pollen', 0.6326375007629395),
 ('sand', 0.6312534809112549)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [26]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted[:1000]]

In [27]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [28]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

C:\Users\Owner\Anaconda3\lib\site-packages\bokeh\models\sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


### Part 5: Wiki Learnt Representations

Download dataset

In [31]:
if not os.path.isfile('wikitext.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [34]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8')

Preprocess sentences (note that it's important to remove small sentences for performance)

In [35]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [44]:
n_sentences_wiki = []
for sent_str in sentences_wiki:
    n_sentences_wiki.append(sent_str.split())

# sample 1/5 of the data
shuffle(n_sentences_wiki)
print(len(n_sentences_wiki))
n_sentences_wiki = n_sentences_wiki[:int(len(n_sentences_wiki)/5)]
print(len(n_sentences_wiki))

853422
170684


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [45]:
freqs_wiki = {}
for sentence in n_sentences_wiki:
    for word in sentence:
        if word in freqs_wiki:
            freqs_wiki[word] += 1
        else:
            freqs_wiki[word] = 1

In [46]:
words_top_wiki = sorted(freqs_wiki, key=lambda x: int(freqs_wiki[x]), reverse=True)
counts_wiki_top1000 = [freqs_wiki[word] for word in words_top_wiki[:1000]]

print(words_top_wiki[:10])
print(counts_wiki_top1000[:10])

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']
[258248, 109845, 100024, 87446, 79962, 71846, 42743, 32594, 30535, 29047]


In [43]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [62]:
model_wiki = Word2Vec(min_count = 5, size=100, sentences = n_sentences_wiki)

In [52]:
model_wiki.most_similar('woman')

[('child', 0.8956335186958313),
 ('girl', 0.851337730884552),
 ('man', 0.8314140439033508),
 ('friend', 0.7588037252426147),
 ('person', 0.7472477555274963),
 ('lady', 0.7465639114379883),
 ('mother', 0.7442833781242371),
 ('young', 0.7429894208908081),
 ('baby', 0.7332104444503784),
 ('daughter', 0.7266232967376709)]

In [57]:
model_wiki.most_similar('laptop')

[('trademark', 0.8857061862945557),
 ('crass', 0.8449282646179199),
 ('stump', 0.8439292311668396),
 ('displaying', 0.8283067941665649),
 ('microphone', 0.8271535634994507),
 ('necklace', 0.8260799050331116),
 ('paddock', 0.8237367272377014),
 ('arranging', 0.8164933919906616),
 ('gesture', 0.8147735595703125),
 ('baton', 0.8140366673469543)]

In [59]:
model_wiki.most_similar(positive=['cat', 'fish'], negative=['dog'])

[('habitats', 0.8849601149559021),
 ('mammals', 0.8603018522262573),
 ('plants', 0.8598695993423462),
 ('populations', 0.8584595322608948),
 ('amino', 0.8539282083511353),
 ('acids', 0.8453362584114075),
 ('bacteria', 0.8453258872032166),
 ('blacktip', 0.845224142074585),
 ('vegetation', 0.8433995842933655),
 ('wildlife', 0.843356728553772)]

#### t-SNE visualization

In [64]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki[:1000]]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [65]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

C:\Users\Owner\Anaconda3\lib\site-packages\bokeh\models\sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
